In [ ]:
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the pre-trained model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/nli-deberta-v3-xsmall')
tokenizer = AutoTokenizer.from_pretrained('cross-encoder/nli-deberta-v3-xsmall')

def process_json(input_file, output_file):
    # Load the JSON data
    with open(input_file, 'r') as f:
        data = json.load(f)

    # Define the label mapping
    label_mapping = ['contradiction', 'entailment', 'neutral']

    # Initialize a counter for the items
    item_number = 0

    # Process each item in the JSON
    for item in data:
        # Increment the item counter
        item_number += 1

        # Extract the premise and hypothesis from the JSON item
        premise = item['Passage']
        hypothesis = item['Question']

        # Tokenize the premise and hypothesis
        features = tokenizer(premise, hypothesis, padding=True, truncation=True, return_tensors="pt")

        # Perform inference using the pre-trained model
        model.eval()
        with torch.no_grad():
            logits = model(**features).logits
            prediction = label_mapping[logits.argmax(dim=1).item()]

        # Add the 'Relation' key with the model's prediction
        item['Relation'] = prediction

        # Print the item number for progress tracking
        print(f"Item number: {item_number}")

    # Save the modified data to a new JSON file
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

# Replace the following with your specific filename and folder paths
filename = "YourDocumentName"

# Specify the input and output file paths
input_file = f'RawQuestions/{filename}_questions.json'
output_file = f'SilverQuestions/{filename}_QA_NLI.json'

# Process the JSON file
process_json(input_file, output_file)
